Инструкция по сбору вопросника:<br>
1. Запускаем  первую ячейку (ctrl+enter)
2. Во второй ячейке проверяем, есть ли в переменной columns название колонки с сорсом, если нет - дописываем через запятую, либо (если это не англ), удаляем все, что там есть и оставляем новое  в кавычках
3. Во второй ячейке проверяем, что в qa_sheet_name  правильно указано название вкладки (меняем на нужную)
4. Запускаем вторую ячейку
5. Запускаем третью ячейку
6. В открывшейся форме указываем, где хранится файл qa и локит
7. После окончания процесса нажимаем значок папки слева, ищем там в папке content итоговый файл


🪓🪓🪓 Ячейку ниже не исправляем, запускаем ее 1 раз! 🪓🪓🪓


In [ ]:
import openpyxl
import time
import re
import os
from google.colab import files
from ipywidgets import widgets

# Function to load a workbook with error handling
def load_workbook_safe(file_path):
  try:
    if not os.path.exists(file_path):
      raise FileNotFoundError(f"File {file_path} does not exist.")
    return openpyxl.load_workbook(file_path)
  except Exception as e:
        print(f"Error loading file {file_path}: {e}")
        return None

# Function to copy headers from LOCKIT to QA
def copy_headers(lockit_sheet, qa_sheet, matching_col_index):
    # Get headers starting from the matching "EN" column
    lockit_headers = [cell.value for cell in lockit_sheet[1][matching_col_index - 1:]]

    # Copy the "EN" header into C1 (column 3) of the QA sheet
    qa_sheet.cell(row=1, column=3, value=lockit_headers[0])

    # Copy the rest of the headers to QA sheet starting from D1
    for col_offset, header in enumerate(lockit_headers[1:], start=4):
        qa_sheet.cell(row=1, column=col_offset, value=header)
    print("Headers copied to QA sheet.")

# Function to find the matching column in LOCKIT
def find_matching_column(lockit_sheet, search_columns):
    # Get column names from the first row of the sheet
    lockit_columns = [cell.value.lower() if cell.value else '' for cell in lockit_sheet[1]]

    # Search for the matching column
    for i, col_name in enumerate(lockit_columns):
        if col_name in search_columns:
            return i + 1  # Return the column index (1-based)
    return None

# Function to process rows in QA and LOCKIT
def process_rows(lockit_sheet, qa_sheet, matching_col_index):
    qa_row = 1
    empty_qa_count = 0

    # Process rows in QA
    while empty_qa_count < 5:
        qa_value = qa_sheet.cell(row=qa_row, column=3).value
        if not qa_value:
            empty_qa_count += 1
        else:
            empty_qa_count = 0

            # Clean and normalize the value
            qa_value_clean = re.sub(r'\s+', ' ', str(qa_value).strip().lower())

            # Search for a match in the current LOCKIT sheet
            lockit_row = 1
            empty_lockit_count = 0

            while empty_lockit_count < 10:
                lockit_value = lockit_sheet.cell(row=lockit_row, column=matching_col_index).value
                if not lockit_value:
                    empty_lockit_count += 1
                else:
                    empty_lockit_count = 0
                    lockit_value_clean = re.sub(r'\s+', ' ', str(lockit_value).strip().lower())
                    if lockit_value_clean == qa_value_clean:
                        # Copy the rest of the row
                        lockit_row_data = [cell.value for cell in lockit_sheet[lockit_row][matching_col_index:]]
                        for col_offset, value in enumerate(lockit_row_data, start=4):
                            qa_sheet.cell(row=qa_row, column=col_offset, value=value)
                        break
                lockit_row += 1
        qa_row += 1

# Main function to process the files
def main(qa_file_path, lockit_file_path, qa_sheet_name, search_columns):
    # Load the workbooks
    qa_wb = load_workbook_safe(qa_file_path)
    lockit_wb = load_workbook_safe(lockit_file_path)

    # Check if the workbooks were loaded successfully
    if qa_wb is None:
        print(f"Failed to load QA file: {qa_file_path}")
        return
    if lockit_wb is None:
        print(f"Failed to load LOCKIT file: {lockit_file_path}")
        return

    # Get the specified sheet in QA
    if qa_sheet_name in qa_wb.sheetnames:
        qa_sheet = qa_wb[qa_sheet_name]
    else:
        print(f"Sheet '{qa_sheet_name}' not found in QA file.")
        return

    # Process each sheet in LOCKIT
    for lockit_sheet in lockit_wb.worksheets:
        matching_col_index = find_matching_column(lockit_sheet, search_columns)
        if matching_col_index is None:
            continue  # Move to the next sheet

        print(f"Matching Column in LOCKIT: {matching_col_index}, {lockit_sheet}")

        # Copy headers to QA
        copy_headers(lockit_sheet, qa_sheet, matching_col_index)

        # Process rows
        process_rows(lockit_sheet, qa_sheet, matching_col_index)

    # Save the updated file
    output_file_path = os.path.join(os.path.dirname(qa_file_path), 'updated_qa.xlsx')
    try:
        qa_wb.save(output_file_path)
        print(f"File saved successfully at {output_file_path}")
    except Exception as e:
        print(f"Error saving file: {e}")
        exit()

def upload_file(prompt):
    print(prompt)
    uploaded = files.upload()
    if not uploaded:
        print("Файл не был загружен. Пожалуйста, попробуйте снова.")
        return None
    return list(uploaded.keys())[0]

🪓 В ячейке ниже проверяем название колонки с сорсом и название вкладки документа QA, после чего запускаем! 🪓

In [ ]:
# Указываем колонку с сорсом (тут перечислены варианты названий английского - это все можно убрать или оставить)
columns = ['en', 'english', 'english (us)', 'english (en-gb)', 'proofread en']

# УКАЖИТЕ НАЗВАНИЕ ВКЛАДКИ В QA ДОКУМЕНТЕ
qa_sheet_name = "PROJECT_NAME DD.MM.YYYY"

🪓 Ячейку ниже просто запускаем, после чего подгружаем нужные файлы в форму 🪓

In [ ]:
qa_file_name = upload_file("Пожалуйста, загрузите файл QA.")
if not qa_file_name:
    print("Ошибка: QA файл не был загружен.")
    exit()

lockit_file_name = upload_file("Пожалуйста, загрузите файл LOCKIT.")
if not lockit_file_name:
    print("Ошибка: LOCKIT файл не был загружен.")
    exit()

print(f"QA файл: {qa_file_name}")
print(f"LOCKIT файл: {lockit_file_name}")

main(qa_file_name, lockit_file_name, qa_sheet_name, columns)

Пожалуйста, загрузите файл QA.


Saving test_qa_archlife.xlsx to test_qa_archlife (3).xlsx
Пожалуйста, загрузите файл LOCKIT.


Saving test_lockit_archlife.xlsx to test_lockit_archlife (3).xlsx
QA файл: test_qa_archlife (3).xlsx
LOCKIT файл: test_lockit_archlife (3).xlsx


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Matching Column in LOCKIT: 11, <Worksheet "KDL - KDL">
Headers copied to QA sheet.
Matching Column in LOCKIT: 2, <Worksheet "Brand names">
Headers copied to QA sheet.
File saved successfully at updated_qa.xlsx
